In [2]:
#출처: https://wikidocs.net/30708
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [5]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print('샘플의 수 :', len(documents))

샘플의 수 : 11314


In [6]:
documents[1]
#뉴스그룹 데이터에는 특수문자가 포함된 다수의 영어 문장으로 구성되어 있고 이런 형식의 샘플이 총 11,314개 존재함

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [7]:
print(dataset.target_names)
#본래의 카테고리

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [15]:
#기본적인 아이디어로 텍스트 전처리(구두점, 숫자, 특수 문자 제거)
news_df = pd.DataFrame({'document':documents})
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [16]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy your logic runs steam sorry pity sorry that have these feelings denial about faith need well just pretend that will happily ever after anyway maybe start newsgroup atheist hard bummin much forget your flintstone chewables bake timmons'

In [17]:
#NLTK로부터 불용어 받아오기
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) #토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
#불용어 제거

In [18]:
print(tokenized_doc[1])

['yeah', 'expect', 'people', 'read', 'actually', 'accept', 'hard', 'atheism', 'need', 'little', 'leap', 'faith', 'jimmy', 'logic', 'runs', 'steam', 'sorry', 'pity', 'sorry', 'feelings', 'denial', 'faith', 'need', 'well', 'pretend', 'happily', 'ever', 'anyway', 'maybe', 'start', 'newsgroup', 'atheist', 'hard', 'bummin', 'much', 'forget', 'flintstone', 'chewables', 'bake', 'timmons']


In [19]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

In [20]:
#정수 인코딩을 동시에 단어의 빈도수를 기록
#word_id 정수인코딩된 값, word_frequency 단어의 빈도수

from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) #수행된 결과에서 두번째 뉴스 출력(첫번째 인덱스 0)
#(66,2)는 정수 인코딩이 66으로 할당된 단어가 두번째 뉴스에서 두번 등장하였음을 의미

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [21]:
#정수 인코딩 전 어떤 단어였는지 파악해보기
print(dictionary[66])

faith


In [22]:
#총 학습된 단어의 개수를 알아뵈 위해 dictionary의 길이 파악

len(dictionary)

64281

In [24]:
#LDA 모델 훈련시키기

import gensim
NUM_TOPICS = 20 #k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)
#총 20개의 토픽이 0-19번으로 할당
#각 단어 앞에 붙은 수치는 단어의 해당 토픽에 대한 기여도
#passes는 알고리즘 동작 횟수: 알고리즘이 결정하는 토픽 값이 적절히 수렴 가능한 충분히 적당한 횟수를 주면 됨
#num_words=4로 총 4개의 단어만 출력

(0, '0.017*"sale" + 0.013*"power" + 0.012*"wire" + 0.012*"condition"')
(1, '0.014*"drive" + 0.013*"windows" + 0.013*"system" + 0.012*"thanks"')
(2, '0.039*"health" + 0.021*"medical" + 0.015*"disease" + 0.015*"cancer"')
(3, '0.009*"picture" + 0.008*"simms" + 0.007*"model" + 0.006*"computer"')
(4, '0.020*"church" + 0.011*"pope" + 0.009*"marriage" + 0.009*"orthodox"')
(5, '0.014*"would" + 0.014*"people" + 0.011*"think" + 0.008*"believe"')
(6, '0.005*"good" + 0.005*"book" + 0.004*"water" + 0.004*"many"')
(7, '0.035*"jpeg" + 0.027*"printer" + 0.025*"scsi" + 0.016*"print"')
(8, '0.012*"government" + 0.009*"public" + 0.007*"encryption" + 0.006*"system"')
(9, '0.020*"armenian" + 0.015*"turkish" + 0.014*"armenians" + 0.010*"greek"')
(10, '0.022*"file" + 0.012*"available" + 0.011*"program" + 0.010*"files"')
(11, '0.022*"gordon" + 0.021*"pitt" + 0.019*"banks" + 0.018*"soon"')
(12, '0.019*"space" + 0.007*"nasa" + 0.007*"research" + 0.006*"program"')
(13, '0.018*"israel" + 0.012*"israeli" + 0.012*"

In [25]:
print(ldamodel.print_topics())

[(0, '0.017*"sale" + 0.013*"power" + 0.012*"wire" + 0.012*"condition" + 0.012*"ground" + 0.011*"offer" + 0.010*"radio" + 0.009*"asking" + 0.009*"current" + 0.008*"used"'), (1, '0.014*"drive" + 0.013*"windows" + 0.013*"system" + 0.012*"thanks" + 0.011*"card" + 0.010*"disk" + 0.009*"know" + 0.009*"problem" + 0.009*"anyone" + 0.008*"please"'), (2, '0.039*"health" + 0.021*"medical" + 0.015*"disease" + 0.015*"cancer" + 0.013*"diseases" + 0.013*"cubs" + 0.013*"patients" + 0.010*"aids" + 0.010*"april" + 0.009*"hicnet"'), (3, '0.009*"picture" + 0.008*"simms" + 0.007*"model" + 0.006*"computer" + 0.006*"navy" + 0.006*"gateway" + 0.005*"byte" + 0.005*"macs" + 0.005*"sleeve" + 0.004*"vram"'), (4, '0.020*"church" + 0.011*"pope" + 0.009*"marriage" + 0.009*"orthodox" + 0.008*"catholic" + 0.008*"luke" + 0.007*"romans" + 0.006*"married" + 0.006*"forsale" + 0.006*"deck"'), (5, '0.014*"would" + 0.014*"people" + 0.011*"think" + 0.008*"believe" + 0.007*"many" + 0.007*"question" + 0.007*"true" + 0.007*"well

In [27]:
#LDA시각화
!pip install pyLDAvis

Looking in indexes: http://ftp.daumkakao.com/pypi/simple
     |████████████████████████████████| 1.7 MB 2.5 MB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached sklearn-0.0-py2.py3-none-any.whl
     |████████████████████████████████| 1.7 MB 19.1 MB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     |████████████████████████████████| 1.7 MB 2.2 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.2.2-py2.py3-none-any.whl size=135591 sha256=9dae73aa6f7a8039a6d77a45c632402e4307fad0aaed95d54e0a425fa8f2f78c
  Stored in directory: /home/ubuntu/.cache/pip/wheels/7a/f9/dd/4171882ef2eb161c41ee7ff06759e2978ec04e13445b0662d1
Successfully built p

In [29]:
import  pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis =  pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

In [30]:
#문서 별 토픽 분포 확인
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5: #i번째 문서의 토픽 비율까지만 출력
        break
    print(i,'번째 문서의 topic 비율은', topic_list)

0 번째 문서의 topic 비율은 [(5, 0.19789329), (9, 0.066005796), (13, 0.47728008), (18, 0.24590532)]
1 번째 문서의 topic 비율은 [(5, 0.78008664), (6, 0.028726293), (15, 0.05849182), (16, 0.111594886)]
2 번째 문서의 topic 비율은 [(5, 0.17418054), (6, 0.04568928), (13, 0.3183156), (16, 0.44867897)]
3 번째 문서의 topic 비율은 [(3, 0.051245306), (5, 0.051378597), (8, 0.25312823), (15, 0.071664795), (16, 0.39736018), (18, 0.16425614)]
4 번째 문서의 topic 비율은 [(5, 0.48669994), (10, 0.07143882), (14, 0.35139787), (18, 0.060817227)]


In [31]:
#데이터 프레임 형식으로 출력
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()
    
    #몇 번째 문서인지 의미하는 무서 번호와 해당 토픽 비중을 한 줄씩 추출
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list
        doc = sorted(doc, key=lambda x: (x[1]), reserve=True)
        #각 문서에 대해 비중이 높은 토픽 순으로 토픽 정렬
        #ex. 2번토픽:50%, 5번토픽:10%, 7번토픽:40% =>2번토픽:50%, 7번토픽:40%, 5번토픽:10% 
        
        #모든 문서에 대해 각각 하단코드 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #몇 번 토픽인지와 비중을 나눠서 저장
            if j==0:#정렬 후 이므로 0일 때 비중이 제일 높음
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                #가장 비중이 높은 토픽과, 그 토픽의 비중과, 전체 토픽의 비중을 저장
            else:
                break
    return(topic_table)

In [32]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictble.reset_index() #문서 번호를 의미하는 열로 사용하기 위해 인덱스 열을 하나 더 만듦
topictable.columns = ['문서번호', '가장 비중이 높은 토픽','토픽의 비중', '각 토픽의 비중']
topictable[:10]

TypeError: 'reserve' is an invalid keyword argument for this function